<a href="https://colab.research.google.com/github/likitha888/python/blob/main/image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow numpy matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

In [3]:
model = VGG16(weights='imagenet')
model.trainable = False  # Freeze the model

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [5]:
def integrated_gradients(model, x, baseline=None, steps=50):
    if baseline is None:
        baseline = tf.zeros_like(x)  # Create a baseline of zeros as a TensorFlow tensor
             # Calculate gradients
    scaled_inputs = [baseline + (i / steps) * (x - baseline) for i in range(steps + 1)]

    # Reshape scaled_inputs to match expected input shape of VGG16
    scaled_inputs = tf.concat(scaled_inputs, axis=0)  # Concatenate along batch dimension

    with tf.GradientTape() as tape:
        tape.watch(scaled_inputs)
        predictions = model(scaled_inputs)

    grads = tape.gradient(predictions, scaled_inputs)

    # Reshape grads to match the shape of scaled_inputs
    grads = tf.reshape(grads, [steps + 1] + list(x.shape))

    avg_grads = tf.reduce_mean(grads[:-1], axis=0)  # Average the gradients
     # Compute the integrated gradients
    integrated_grads = (x - baseline) * avg_grads
    return integrated_grads.numpy() # Convert to NumPy array for visualization

In [6]:
def visualize_attributions(img_path, attributions):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img) / 255.0  # Normalize the image

    # Squeeze the attributions array to remove the batch dimension
    attributions = np.squeeze(attributions)

    attributions = np.abs(attributions)  # Use absolute values for visualization
    attributions = attributions / np.max(attributions)  # Normalize attributions

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(img)

    plt.subplot(1, 2, 2)
    plt.title('Attributions')
    plt.imshow(attributions, cmap='hot')
    plt.show()
